## starting point in SparkSql is SparkSession
The entry point into all functionality in Spark is the SparkSession class. To create a basic SparkSession, just use SparkSession.builder:

In [1]:
from pyspark.sql import SparkSession


In [2]:
spark = SparkSession.builder.appName("MDs example of Spark SQL").config("spark.some.config.option", "some-value").getOrCreate()

*SparkSession* provides support for Appache Hive, but Appache Hive would need to be installed.

Following are some examples from GitHub Spark examples.
Reference:
https://github.com/apache/spark/blob/master/examples/src/main/python/sql/basic.py

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import *


In [5]:

df = spark.read.json("./data/people.json")
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [6]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



select something, first we select the column *name*, then select everybody and increment the age with 1

In [8]:
df.select("name").show()

+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
+-------+



In [10]:
ret = df.select(df["name"], df["age"] + 1)
ret.show()

+-------+---------+
|   name|(age + 1)|
+-------+---------+
|Michael|     null|
|   Andy|       31|
| Justin|       20|
+-------+---------+



select with some criteria clause, select people older than 25 years

In [11]:
ret = df.filter(df["age"] >= 25)
ret.show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



group people by age and count them

In [14]:
df.groupby("age").count().show()

+----+-----+
| age|count|
+----+-----+
|  19|    1|
|null|    1|
|  30|    1|
+----+-----+



## temporary and global temporary view

Temporary views in Spark SQL are session-scoped and will disappear if the session that creates it terminates.<br>
If you want to have a temporary view that is shared among all sessions and keep alive until the Spark application terminates, you can create a global temporary view.<br>

Global temporary view is tied to a system preserved database **global_temp**, and we must use the qualified name to refer it, e.g. *SELECT * FROM global_temp.view1*.

register the DataFrame as a SQL temporary view<br>
not a global temporary view.

In [17]:
df.createOrReplaceTempView("people")

In [18]:
sqlDF = spark.sql("select * from people")
sqlDF.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



register the DataFrame as a Global temp view<br>
in  this case it gets tied to a system preserved database called **global_temp**

In [20]:
df.createOrReplaceGlobalTempView("people")

In [21]:
spark.sql("select * from global_temp.people").show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



the global temp view is persisting across sessions<br>
here we create a new session and show that we can query people in the new session

In [22]:
spark.newSession().sql("select * from global_temp.people").show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



## schema inference example

Spark SQL supports two different methods for converting existing RDDs into Datasets. The first method uses reflection to infer the schema of an RDD that contains specific types of objects. This reflection based approach leads to more concise code and works well when you already know the schema while writing your Spark application.

The second method for creating Datasets is through a programmatic interface that allows you to construct a schema and then apply it to an existing RDD. While this method is more verbose, it allows you to construct Datasets when the columns and their types are not known until runtime.


get the spark context from the session<br>
then load a text file and convert each line to a Row (we imported Row)<br>
then infer the schema and register the DF as a table<br>
then we run some SQL statements

In [23]:
sc = spark.sparkContext

In [24]:
lines = sc.textFile("./data/people.txt")

In [24]:
parts = lines.map(lambda l: l.split(","))

In [25]:
people = parts.map(lambda p: Row(name=p[0], age=int(p[1])))

infer the schema and register the DataFrame as a table

In [26]:
schemaP = spark.createDataFrame(people)
schemaP.createOrReplaceTempView("people")

nowm run some SQL

In [27]:
teenagers = spark.sql("select name from people where age >= 13 and age <= 19")
teenagers.show()

+------+
|  name|
+------+
|Justin|
+------+



the result of sql queries is of type DataFrame<br>
.rdd will return the content as RDD: *pyspark.RDD* of *class:"Row"*

In [30]:
teenNames = teenagers.rdd.map(lambda p: "Name: " + p.name).collect()
type(teenNames)

list

In [31]:
for name in teenNames:
    print(name)

Name: Justin


## programmatic schema example

When a dictionary of kwargs cannot be defined ahead of time (for example, the structure of records is encoded in a string, or a text dataset will be parsed and fields will be projected differently for different users), a DataFrame can be created programmatically with three steps.

1. Create an RDD of tuples or lists from the original RDD;
2. Create the schema represented by a *StructType* matching the structure of tuples or lists in the RDD created in the step 1.
3. Apply the schema to the RDD via *createDataFrame* method provided by *SparkSession*.

we apply here a schema to the RDD obtained from a text file<br>
then we run a sql query on it, because we can.

In [32]:
people = parts.map(lambda p: (p[0], p[1].strip()))

In [33]:
schemaString = "name age"

In [34]:
fields = [StructField(field_name, StringType(), True) for field_name in schemaString.split()]
fields

[StructField(name,StringType,true), StructField(age,StringType,true)]

In [35]:
schema = StructType(fields)
schema

StructType(List(StructField(name,StringType,true),StructField(age,StringType,true)))

apply the schema to the RDD

In [36]:
schemaPeople = spark.createDataFrame(people, schema)

create a temp view out of schemaPeople so we can run sql queries on it

In [37]:
schemaPeople.createOrReplaceTempView("people")
spark.sql("SELECT name FROM people").show()

+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
+-------+



## aggregations

The built-in DataFrames functions provide common aggregations such as *count()*, *countDistinct()*, *avg()*, *max()*, *min()*, etc. While those functions are designed for *DataFrames*, Spark SQL also has type-safe versions for some of them in Scala and Java to work with strongly typed Datasets. Moreover, users are not limited to the predefined aggregate functions and can create their own.

see Sp_02_SparkSql_Scala.ipynb notebook in scala for an example.


## data sources

Spark SQL supports operating on a variety of data sources through the DataFrame interface. A DataFrame can be operated on using relational transformations and can also be used to create a temporary view. Registering a DataFrame as a temporary view allows you to run SQL queries over its data. This section describes the general methods for loading and saving data using the Spark Data Sources and then goes into specific options that are available for the built-in data sources.

In the simplest form, the default data source (parquet unless otherwise configured by spark.sql.sources.default) will be used for all operations.

In [38]:
df = spark.read.load("./data/users.parquet")
df.select("name", "favorite_color").write.save("./data/namesAndFavColors.parquet")

In [39]:
df = spark.read.load("./data/people.json", format="json")
df.select("name", "age").write.save("./data/namesAndAges.parquet", format="parquet")

## run SQL on files directly

In [43]:
df = spark.sql("SELECT * FROM parquet.`./data/users.parquet`")
df.show()


+------+--------------+----------------+
|  name|favorite_color|favorite_numbers|
+------+--------------+----------------+
|Alyssa|          null|  [3, 9, 15, 20]|
|   Ben|           red|              []|
+------+--------------+----------------+



## parquet files

Parquet is a columnar format that is supported by many other data processing systems. Spark SQL provides support for both reading and writing Parquet files that automatically preserves the schema of the original data. When writing Parquet files, all columns are automatically converted to be nullable for compatibility reasons.

In [46]:
peopleDF = spark.read.json("./data/people.json")

# DataFrames can be saved as Parquet files, maintaining the schema information.
peopleDF.write.parquet("./data/people_from_spark.parquet")

# Read in the Parquet file created above.
# Parquet files are self-describing so the schema is preserved.
# The result of loading a parquet file is also a DataFrame.
parquetFile = spark.read.parquet("./data/people_from_spark.parquet")

# Parquet files can also be used to create a temporary view and then used in SQL statements.
parquetFile.createOrReplaceTempView("parquetFile")
teenagers = spark.sql("SELECT name FROM parquetFile WHERE age >= 13 AND age <= 19")
teenagers.show()

+------+
|  name|
+------+
|Justin|
+------+



## partition discovery

Table partitioning is a common optimization approach used in systems like Hive. In a partitioned table, data are usually stored in different directories, with partitioning column values encoded in the path of each partition directory. The Parquet data source is now able to discover and infer partitioning information automatically. For example, we can store all our previously used population data into a partitioned table using the following directory structure, with two extra columns, gender and country as partitioning columns:


see:
https://spark.apache.org/docs/2.2.0/sql-programming-guide.html

By passing path/to/table to either SparkSession.read.parquet or SparkSession.read.load, Spark SQL will automatically extract the partitioning information from the paths. Now the schema of the returned DataFrame becomes:

## parquet schema merging

Like ProtocolBuffer, Avro, and Thrift, Parquet also supports schema evolution. Users can start with a simple schema, and gradually add more columns to the schema as needed. In this way, users may end up with multiple Parquet files with different but mutually compatible schemas. The Parquet data source is now able to automatically detect this case and merge schemas of all these files.

In [47]:
from pyspark.sql import Row

# spark is from the previous example.
# Create a simple DataFrame, stored into a partition directory
sc = spark.sparkContext

squaresDF = spark.createDataFrame(sc.parallelize(range(1, 6))
                                  .map(lambda i: Row(single=i, double=i ** 2)))
squaresDF.write.parquet("./data/test_table/key=1")

# Create another DataFrame in a new partition directory,
# adding a new column and dropping an existing column
cubesDF = spark.createDataFrame(sc.parallelize(range(6, 11))
                                .map(lambda i: Row(single=i, triple=i ** 3)))
cubesDF.write.parquet("./data/test_table/key=2")

# Read the partitioned table
mergedDF = spark.read.option("mergeSchema", "true").parquet("data/test_table")
mergedDF.printSchema()

root
 |-- double: long (nullable = true)
 |-- single: long (nullable = true)
 |-- triple: long (nullable = true)
 |-- key: integer (nullable = true)



## JSON datasets

Spark SQL can automatically infer the schema of a JSON dataset and load it as a DataFrame. This conversion can be done using SparkSession.read.json on a JSON file.

Note that the file that is offered as a json file is not a typical JSON file. Each line must contain a separate, self-contained valid JSON object. For more information, please see JSON Lines text format, also called newline-delimited JSON.

For a regular multi-line JSON file, set the multiLine parameter to True.


In [48]:
# spark is from the previous example.
sc = spark.sparkContext

# A JSON dataset is pointed to by path.
# The path can be either a single text file or a directory storing text files
path = "data/people.json"
peopleDF = spark.read.json(path)

# The inferred schema can be visualized using the printSchema() method
peopleDF.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [49]:
# Creates a temporary view using the DataFrame
peopleDF.createOrReplaceTempView("people")

# SQL statements can be run by using the sql methods provided by spark
teenagerNamesDF = spark.sql("SELECT name FROM people WHERE age BETWEEN 13 AND 19")
teenagerNamesDF.show()

+------+
|  name|
+------+
|Justin|
+------+



## stop Spark session here

In [51]:
spark.stop()